In [ ]:
#NOTE: This cell performs spectral modelling and fitting without taking into account CASTOR's instrumentation files

#import packages for spectral modelling - Sherpa
import numpy as np
import matplotlib.pyplot as plt
import sherpa.astro.ui as ui
from sherpa.models.basic import Gauss1D, PowLaw1D
from sherpa.data import Data1D
from sherpa.plot import DataPlot, ModelPlot, FitPlot
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar, NelderMead
from sherpa.fit import Fit
from astropy import units as u

#create AGN model

#create the power law continuum model
#set initial photon index of power law to 2
powerlaw = PowLaw1D()
powerlaw.gamma = -1.5

#note redshift has not been accounted for just yet
#rest wavelengths of gaussian line models (nm)
#need to convert to keV as ARF and RMF are in keV
lyman_wave = 1215.67    
lyman_keV = (lyman_wave * u.angstrom).to(u.keV, equivalencies=u.spectral())

mg_wave = 2798.75
mg_keV = (mg_wave * u.angstrom).to(u.keV, equivalencies=u.spectral())

c_wave = 1549.06
c_keV = (c_wave * u.angstrom).to(u.keV, equivalencies=u.spectral())
 
#create the Gaussian line models
#relative parameters for emission lines are taken from Vanden Berk paper
#paper link: https://ui.adsabs.harvard.edu/abs/2001AJ....122..549V/abstract 

#lyman alpha emission line taken at rest wavelength
lyman_alpha = Gauss1D()
lyman_alpha.pos = lyman_keV.value   #Lyman alpha position (keV)
lyman_alpha.ampl = 100/10000        #Lyman alpha amplitude (photon flux)
#obtain lyman alpha fwhm in keV 
lyman_x1 = lyman_wave - (19.46/2)
lyman_x2 = lyman_wave + (19.46/2)
lyman_x1_keV = ((lyman_x1 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
lyman_x2_keV = ((lyman_x2 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
lyman_fwhm = (lyman_x1_keV - lyman_x2_keV)
lyman_alpha.fwhm = lyman_fwhm       #Lyman alpha fwhm (keV)

#Mg[II] emission line taken at rest wavelength
mg_ii = Gauss1D()
mg_ii.pos = mg_keV.value            #Magnesium II position (keV)
mg_ii.ampl = 14.725/10000           #Magnesium II amplitude (photon flux)
#obtain Mg[II] fwhm in keV 
mg_x1 = mg_wave - (34.95/2)
mg_x2 = mg_wave + (34.95/2)
mg_x1_keV = ((mg_x1 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
mg_x2_keV = ((mg_x2 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
mg_fwhm = (mg_x1_keV - mg_x2_keV)
mg_ii.fwhm = mg_fwhm               #Magnesium II fwhm (keV)

#C[IV] emission line taken at rest wavelength
c_iv = Gauss1D()
c_iv.pos = c_keV.value             #Carbon IV position (keV)
c_iv.ampl = 25.291/10000           #Carbon IV amplitude (photon flux)
#obtain C[IV] fwhm in keV 
c_x1 = c_wave - (14.33/2)
c_x2 = c_wave + (14.33/2)
c_x1_keV = ((c_x1 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
c_x2_keV = ((c_x2 * u.angstrom).to(u.keV, equivalencies=u.spectral())).value
c_fwhm = (c_x1_keV - c_x2_keV)
c_iv.fwhm = c_fwhm                 #Carbon IV fwhm (keV)

#define the model 
#final model contains all emission lines present in AGN source 
#emission lines are added to the powerlaw
AGN_model = lyman_alpha + mg_ii + c_iv + powerlaw